<a href="https://colab.research.google.com/github/iplusplus42/haiku-gen/blob/main/Haiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# NOTE: don't run this cell on Colab, it might break something. It loads my code formatter locally.
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [16]:
import random
import os
from urllib import request
import zipfile
from typing import List, Tuple

DEBUG = False
LOCAL = os.path.exists(os.path.join(os.getcwd(), ".gitignore"))


class Utils:
    @staticmethod
    def is_palindrome(string, ignore_whitespace=True):
        if ignore_whitespace:
            string = string.replace(" ", "")
        return string == string[::-1]

    @staticmethod
    def get_files():
        with request.urlopen("https://insertplus.keybase.pub/words.zip") as f:
            fd = open("words.zip", "wb")
            fd.write(f.read())
            fd.close()

            fd = open("words.zip", "rb")
            words_zip = zipfile.ZipFile(fd)
            words_zip.extractall()
            words_zip.close()
            fd.close()

    @staticmethod
    def get_adjectives(syllable=1):
        adjectives = set()
        path = os.path.join(os.getcwd(), "words/adjectives")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableadjectives.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            adjectives.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                adjectives.update({i for i in words})
                fd.close()
        return adjectives

    @staticmethod
    def get_adj(syllable=1):
        return random.choice([i for i in Utils.get_adjectives(syllable)])

    get_adjective = get_adj

    @staticmethod
    def get_adverbs(syllable=1):
        adverbs = set()
        path = os.path.join(os.getcwd(), "words/adverbs")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableadverbs.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            adverbs.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                adverbs.update({i for i in words})
                fd.close()
        return adverbs

    @staticmethod
    def get_adverb(syllable=1):
        return random.choice([i for i in Utils.get_adverbs(syllable)])

    @staticmethod
    def get_nouns(syllable=1):
        nouns = set()
        path = os.path.join(os.getcwd(), "words/nouns")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllablenouns.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            nouns.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                nouns.update({i for i in words})
                fd.close()
        return nouns

    @staticmethod
    def get_noun(syllable=1):
        return random.choice([i for i in Utils.get_nouns(syllable)])

    @staticmethod
    def get_verbs(syllable=1):
        verbs = set()
        path = os.path.join(os.getcwd(), "words/verbs")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableverbs.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            verbs.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                verbs.update({i for i in words})
                fd.close()
        return verbs

    @staticmethod
    def get_verb(syllable=1):
        return random.choice([i for i in Utils.get_verbs(syllable)])

    @staticmethod
    def get_words(syllable=1):
        return random.choice([Utils.get_verbs, Utils.get_nouns, Utils.get_adverbs, Utils.get_adjectives])(syllable)

    @staticmethod
    def get_word(syllable=1):
        return random.choice(
            [
                i
                for i in random.choice([Utils.get_verbs, Utils.get_nouns, Utils.get_adverbs, Utils.get_adjectives])(
                    syllable
                )
            ]
        )


class Rules:
    @staticmethod
    def do_element_rule(elements: List[str], total_syllables: int):
        if len(elements) > total_syllables:
            raise IndexError("elements list cannot be longer than total_syllables!")
        syllables = [{"name": i, "count": 1} for i in elements]
        syllable_bank = total_syllables - len(elements)
        while syllable_bank:
            syllables[random.randrange(len(elements))]["count"] += 1
            syllable_bank -= 1
        debug(f"element rule syllables{syllables}")
        line = ""
        for i in range(len(elements)):
            line += f"{getattr(Utils, f'get_{elements[i]}')(syllables[i]['count'])} "
        return line

    @staticmethod
    def _do_word_length_rule(element: str, syllables: int, length: Tuple[str, int] = ("eq", 7), breakout=20):
        if syllables not in range(1, 5):
            raise RuntimeError("syllables must be between 1 and 4!")
        for _i in range(breakout):
            word = getattr(Rules, f"get_{element}")(syllables)
            word_length = len(word)
            (operator, value) = length
            if (
                (operator == "eq" and word_length == value)
                or (operator == "lt" and word_length < value)
                or (operator == "gt" and word_length > value)
                or (operator == "le" and word_length <= value)
                or (operator == "ge" and word_length >= value)
            ):
                return word
        return InterruptedError(f"Was unable to meet the rule in the iterations requested: {breakout}")

    @staticmethod
    def do_line_length_rule(
        elements: List[str], total_syllables: int, length: Tuple[str, int] = ("eq", 20), breakout=20
    ):
        for _i in range(breakout):
            line = Rules.do_element_rule(elements, total_syllables)
            line_length = len(line)
            (operator, value) = length
            if (
                (operator == "eq" and line_length == value)
                or (operator == "lt" and line_length < value)
                or (operator == "gt" and line_length > value)
                or (operator == "le" and line_length <= value)
                or (operator == "ge" and line_length >= value)
            ):
                return line
        return InterruptedError(f"Was unable to meet the rule in the iterations requested: {breakout}")

    @staticmethod
    def random_rule(total_syllables: int):
        rule = random.choice([i for i in Rules().__dir__() if i.startswith("do_")])
        elements = ["adj", "noun", "verb"]
        random_elements = []
        for i in range(random.randint(2, total_syllables)):
            random_elements.append(random.choice(elements))
        return getattr(Rules, rule)(random_elements, total_syllables)


def debug(content):
    if DEBUG:
        print(content)


def absurd_haiku():
    line_1_syllables = 5
    line_2_syllables = 7
    line_3_syllables = 5
    temp = {"line1": "", "line2": "", "line3": ""}

    debug("Line 1")
    while line_1_syllables > 0:
        syllable = random.randint(1, 4 if line_1_syllables > 4 else line_1_syllables)
        debug(f"Remaining syllables: {line_1_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_1_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line1"] += f"{word} "
        line_1_syllables -= syllable

    debug("Line 2")
    while line_2_syllables > 0:
        syllable = random.randint(1, 4 if line_2_syllables > 4 else line_2_syllables)
        debug(f"Remaining syllables: {line_2_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_2_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line2"] += f"{word} "
        line_2_syllables -= syllable

    debug("Line 3")
    while line_3_syllables > 0:
        syllable = random.randint(1, 4 if line_3_syllables > 4 else line_3_syllables)
        debug(f"Remaining syllables: {line_3_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_3_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line3"] += f"{word} "
        line_3_syllables -= syllable

    return f"{temp['line1']}\n{temp['line2']}\n{temp['line3']}"


class InteractiveMenu:
    def main(self):
        last_input = ""
        while last_input != "q":
            print(
                """
            1) Generate haiku
            2) View haikus
            q) Quit
            """
            )
            last_input = input()
            if last_input == "1":
                self.generate_haiku_menu()

    def generate_haiku_menu(self):
        last_input = ""
        while last_input != "b":
            print(
                """
            1) Select elements
            2) Select rules

            r) Random haiku
            b) Back
            """
            )
            last_input = input()
            if last_input == "1":
                self.select_elements_menu()

    def select_elements_menu(self):
        last_input = ""
        while last_input != "b":
            print(
                """
            1) Adjective
            2) Adverb
            3) Noun
            4) Verb

            u) Undo
            c) Clear
            b) Back
            """
            )
            last_input = input()


if __name__ == "__main__":
    if not LOCAL:
        Utils.get_files()
    print("HaikuGenerator v0.2.0")
    # menu = InteractiveMenu()
    # menu.main()
    print("Absurd haiku")
    print(absurd_haiku())
    print("\nAdjective noun verb rule")
    print(Rules.do_element_rule(["adj", "noun", "verb"], 5))
    print(Rules.do_element_rule(["adj", "noun", "verb"], 7))
    print(Rules.do_element_rule(["adj", "noun", "verb"], 5))
    print("\nAdjective noun rule")
    print(Rules.do_element_rule(["adj", "noun"], 5))
    print(Rules.do_element_rule(["adj", "noun"], 7))
    print(Rules.do_element_rule(["adj", "noun"], 5))
    print("\nNoun verb rule")
    print(Rules.do_element_rule(["noun", "verb"], 5))
    print(Rules.do_element_rule(["noun", "verb"], 7))
    print(Rules.do_element_rule(["noun", "verb"], 5))
    print("\nVerb adjective noun")
    print(Rules.do_element_rule(["verb", "adj", "noun"], 5))
    print(Rules.do_element_rule(["verb", "adj", "noun"], 7))
    print(Rules.do_element_rule(["verb", "adj", "noun"], 5))
    print("\nRandom rules")
    print(Rules.random_rule(5))
    print(Rules.random_rule(7))
    print(Rules.random_rule(5))
    print("\nLine length rules")
    print(Rules.do_line_length_rule(["adj", "noun", "verb"], 5, ("gt", 10)))
    print(Rules.do_line_length_rule(["verb", "noun"], 7, ("eq", 15)))
    print(Rules.do_line_length_rule(["adj", "verb"], 5, ("lt", 15), 50))

HaikuGenerator v0.2.0
Absurd haiku
sevenfold stump meld 
proconsulates overrules 
locally samphire 

Adjective noun verb rule
backless glare chimneyed 
covinous intrusion spares 
vain gawk discipline 

Adjective noun rule
uranic spigots 
titulary allergies 
speedless latria 

Noun verb rule
sip duplicating 
cyclopropane pluralised 
plexor deciphers 

Verb adjective noun
cleaves far atrophy 
practices hidden surfings 
hucksters frail yeses 

Random rules
Was unable to meet the rule in the iterations requested: 20
lobectomies mustier 
postdate platinized 

Line length rules
tart destriers delve 
Was unable to meet the rule in the iterations requested: 20
cagier store 
